## Model Resnet: modules & parameters

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch.nn as nn
import numpy

class channel_selection(nn.Module):
    """
    Select channels from the output of BatchNorm2d layer. It should be put directly after BatchNorm2d layer.
    The output shape of this layer is determined by the number of 1 in `self.indexes`.
    """
    def __init__(self, num_channels):
        """
        Initialize the `indexes` with all one vector with the length same as the number of channels.
        During pruning, the places in `indexes` which correpond to the channels to be pruned will be set to 0.
        """
        super(channel_selection, self).__init__()
        self.indexes = nn.Parameter(torch.ones(num_channels))

    def forward(self, input_tensor):
        """
        Parameter
        ---------
        input_tensor: (N,C,H,W). It should be the output of BatchNorm2d layer.
        """
        selected_index = np.squeeze(np.argwhere(self.indexes.data.cpu().numpy()))
        if selected_index.size == 1:
            selected_index = np.resize(selected_index, (1,)) 
        output = input_tensor[:, selected_index, :, :]
        return output

In [3]:
import math
import torch.nn as nn
import numpy as np

"""
preactivation resnet with bottleneck design.
"""


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, cfg, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.select = channel_selection(inplanes)
        self.conv1 = nn.Conv2d(cfg[0], cfg[1], kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(cfg[1])
        self.conv2 = nn.Conv2d(cfg[1], cfg[2], kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(cfg[2])
        self.conv3 = nn.Conv2d(cfg[2], planes * self.expansion, kernel_size=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        # group1
        out = self.bn1(x)
        out = self.select(out)
        out = self.relu(out)
        out = self.conv1(out)

        # group2
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv2(out)

        # group3
        out = self.bn3(out)
        out = self.relu(out)
        out = self.conv3(out)

        # down sample
        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        return out

    def forward_bn(self, x):
        bn_value = []
        residual = x

        out = self.bn1(x)
        bn_value.append(out.clone())
        out = self.select(out)
        out = self.relu(out)
        out = self.conv1(out)

        out = self.bn2(out)
        bn_value.append(out.clone())
        out = self.relu(out)
        out = self.conv2(out)

        out = self.bn3(out)
        bn_value.append(out.clone())
        out = self.relu(out)
        out = self.conv3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        return out, bn_value

    def mask_bn(self, index, cfg_mask):
        if index == 0:
            self.bn1.weight.data.mul_(cfg_mask)
            self.bn1.bias.data.mul_(cfg_mask)
        elif index == 1:
            self.bn2.weight.data.mul_(cfg_mask)
            self.bn2.bias.data.mul_(cfg_mask)
        elif index == 2:
            self.bn3.weight.data.mul_(cfg_mask)
            self.bn3.bias.data.mul_(cfg_mask)
        else:
            raise ValueError("Index is not including.")


class resnet(nn.Module):
    def __init__(self, depth=164, dataset='cifar10', cfg=None, conv_cfg=None):
        """
        :param depth:
            164 layers => 1 conv2d + 3 layers × 18 blocks (every layer)  × 3 conv2ds (every block)  + 1 avgPool2d
            param n = (depth - 2) // 9:
                n means how many blocks in every layer
                9 = 3 layers × 3 conv2d (every block)
        :param cfg:
            if depth = 164, then len(cfg) = 164
        :param conv_cfg:
            layer block index examples (index starts at 1 & ≤ 18):
                3 indexes / layer: [4, 9, 14] or [6, 12, 18]
                2 indexes / layer: [9, 18]
                1 index / layer: [18]

        number of BatchNorm2d:
            163 = 162 (3 layers × 18 Bottlenecks × 3 BatchNorm2ds) + 1 BatchNorm2d
        """
        super(resnet, self).__init__()
        assert (depth - 2) % 9 == 0, 'depth should be 9n+2'

        n = (depth - 2) // 9  # depth = 164, n = 18
        block = Bottleneck
        self.block_cfg = conv_cfg
        self.inplanes = 16

        # model config
        if cfg is None:
            cfg = [[16, 16, 16], [64, 16, 16] * (n - 1),
                   [64, 32, 32], [128, 32, 32] * (n - 1),
                   [128, 64, 64], [256, 64, 64] * (n - 1), [256]]
            cfg = [item for sub_list in cfg for item in sub_list]

        # model feature
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1, bias=False)
        self.layer1 = self._make_layer(block, 16, n, cfg=cfg[0: 3 * n])
        self.layer2 = self._make_layer(block, 32, n, cfg=cfg[3 * n: 6 * n], stride=2)
        self.layer3 = self._make_layer(block, 64, n, cfg=cfg[6 * n:9 * n], stride=2)
        self.bn = nn.BatchNorm2d(64 * block.expansion)
        self.select = channel_selection(64 * block.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AvgPool2d(8)

        # model classifier
        if dataset == 'cifar10':
            num_classes = 10
        elif dataset == 'cifar100':
            num_classes = 100
        else:
            raise ValueError('Model `dataset` parameter is Error!')
        self.fc = nn.Linear(cfg[-1], num_classes)

        # model initialize weight
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(0.5)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, cfg, stride=1):
        """
        :param block: Bottleneck item
        :param planes: record the layer's output channel size
                        planes * expansion = output_planes_size,
                        like 16 * 4 = 128 (the first block output)
        :param blocks: number of Bottleneck in layer
        :param cfg:  channel config of all blocks
                        3 cfg items / Bottleneck
        :param stride: default = 1
        """
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False))
        layers = [block(self.inplanes, planes, cfg[0:3], stride, downsample)]
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, cfg[3 * i: 3 * (i + 1)]))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)

        block_value = []
        if self.block_cfg:
            for idx, item in enumerate(self.layer1):
                x = item(x)
                if idx + 1 in self.block_cfg:
                    block_value.append(x)
            for idx, item in enumerate(self.layer2):
                x = item(x)
                if idx + 1 in self.block_cfg:
                    block_value.append(x)
            for idx, item in enumerate(self.layer3):
                x = item(x)
                if idx + 1 in self.block_cfg:
                    block_value.append(x)
        else:
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)

        x = self.bn(x)
        x = self.select(x)
        x = self.relu(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        y = self.fc(x)

        if len(block_value):
            return y, block_value
        return y


In [4]:
import os

def resume_model(resume_file):
    if not os.path.isfile(resume_file):
        raise ValueError("Resume model file is not found at '{}'".format(resume_file))
    print("=> loading checkpoint '{}'".format(resume_file))
    checkpoint = torch.load(resume_file)
    if 'epoch' in checkpoint:
        start_epoch = checkpoint['epoch']
    else:
        start_epoch = None
        
    if 'best_prec1' in checkpoint:
        best_prec1 = checkpoint['best_prec1']
    else:
        best_prec1 = None
        
    if 'state_dict' in checkpoint:
        state_dict = checkpoint['state_dict']
    else:
        state_dict = None
    
    if 'optimizer' in checkpoint:
        opti_dict = checkpoint['optimizer']
    else:
        opti_dict = None
        
    if 'cfg' in checkpoint:
        cfg = checkpoint['cfg']
        print("-> model cfg is loading...\n cfg: {}".format(list(cfg)))
    else:
        cfg = None
        print("-> not found model cfg...")
    print("=>  epoch {} Prec1: {}".format(start_epoch, best_prec1))
    return state_dict, opti_dict, start_epoch, best_prec1, cfg


data_path = {
    'root': r'D:\Project\Pycharm\network-slimming\logs',
    'root2': r'D:\Project\Gitee\network-slimming\logs',
    'bn': [
        'bn_prune_resnet164_cifar100_percent_0.6',
    ],
    'at': [
         'at_prune_resnet164_cifar10_percent_0.4',
         'at_prune_resnet164_cifar100_percent_0.6'
    ],
    'flie': 'model_best.pt',
    'file2': 'model_best.pth.tar',
    'file3': 'pruned.pt',
    'file4': 'pruned.pth.tar'
}

file_path = os.path.join(data_path['root'], data_path['at'][0], data_path['file4'])
state_dict, opti_dict, start_epoch, best_prec1, cfg = resume_model(file_path)

=> loading checkpoint 'D:\Project\Pycharm\network-slimming\logs\at_prune_resnet164_cifar10_percent_0.4\pruned.pth.tar'
-> model cfg is loading...
 cfg: [13, 15, 16, 23, 13, 14, 21, 15, 16, 12, 15, 15, 18, 16, 15, 20, 16, 16, 15, 15, 16, 14, 14, 16, 20, 15, 15, 13, 12, 15, 21, 15, 16, 16, 13, 15, 17, 14, 16, 14, 13, 15, 17, 12, 15, 13, 8, 13, 18, 15, 12, 15, 13, 15, 30, 31, 32, 42, 31, 32, 54, 30, 31, 45, 31, 32, 51, 30, 32, 42, 30, 32, 49, 31, 32, 35, 30, 32, 59, 30, 32, 54, 32, 32, 59, 32, 32, 66, 32, 32, 69, 32, 32, 55, 32, 32, 53, 32, 32, 55, 32, 32, 53, 32, 31, 42, 32, 32, 91, 64, 64, 120, 64, 64, 93, 61, 64, 99, 64, 64, 98, 64, 64, 126, 64, 64, 139, 64, 64, 126, 64, 62, 133, 64, 63, 133, 64, 63, 127, 64, 62, 132, 64, 63, 131, 64, 60, 112, 63, 60, 108, 63, 58, 110, 63, 60, 106, 63, 60, 89, 62, 55, 68]
=>  epoch None Prec1: None


In [5]:
model = resnet(depth=164, dataset='cifar10', cfg=cfg, conv_cfg=None)
model.load_state_dict(state_dict)
model.eval()

RuntimeError: Error(s) in loading state_dict for resnet:
	Missing key(s) in state_dict: "conv1.weight", "layer1.0.bn1.weight", "layer1.0.bn1.bias", "layer1.0.bn1.running_mean", "layer1.0.bn1.running_var", "layer1.0.select.indexes", "layer1.0.conv1.weight", "layer1.0.bn2.weight", "layer1.0.bn2.bias", "layer1.0.bn2.running_mean", "layer1.0.bn2.running_var", "layer1.0.conv2.weight", "layer1.0.bn3.weight", "layer1.0.bn3.bias", "layer1.0.bn3.running_mean", "layer1.0.bn3.running_var", "layer1.0.conv3.weight", "layer1.0.downsample.0.weight", "layer1.1.bn1.weight", "layer1.1.bn1.bias", "layer1.1.bn1.running_mean", "layer1.1.bn1.running_var", "layer1.1.select.indexes", "layer1.1.conv1.weight", "layer1.1.bn2.weight", "layer1.1.bn2.bias", "layer1.1.bn2.running_mean", "layer1.1.bn2.running_var", "layer1.1.conv2.weight", "layer1.1.bn3.weight", "layer1.1.bn3.bias", "layer1.1.bn3.running_mean", "layer1.1.bn3.running_var", "layer1.1.conv3.weight", "layer1.2.bn1.weight", "layer1.2.bn1.bias", "layer1.2.

In [ ]:
from torchstat import stat

stat(model, (3, 32, 32))